In [5]:
import numpy as np
from scipy.spatial import Delaunay
from preprocessing.graph_from_snapshot import build_graph_from_snapshot

In [4]:
# Load static mesh #
mesh_path = "/Users/karan94/Documents/MeshGraphNets_Airfoil/data/Airfoil"
grid = np.load(f"{mesh_path}/test_grid.npy")
cells = np.load(f"{mesh_path}/test_cells.npy")

# Load dynamic data
data = np.load(f"{mesh_path}/test_data.npy")

print(f"Grid shape: {grid.shape}, Cells shape: {cells.shape}, Data shape: {data.shape}")

Grid shape: (100, 5233, 31, 2), Cells shape: (100, 10216, 31, 3), Data shape: (100, 5233, 31, 4)


In [6]:
frame = 0
airfoil_id = 0
features = data[airfoil_id, :, frame, :3]   #
target = data[airfoil_id, :, frame + 1, :3] # next frame
node_pos = grid[airfoil_id, :, frame, :]
triangles = cells[airfoil_id, :, frame, :]
node_type = np.zeros((features.shape[0],)) # dummy

In [7]:
graph = build_graph_from_snapshot(node_pos, triangles, features, target, node_type)

In [8]:
graph

{'nodes': array([[ 1.0473521e+02,  1.3460818e+01,  1.2269930e+00,  9.9975002e-01,
         -3.6328966e-05,  0.0000000e+00],
        [ 1.1291812e+02,  1.6061655e+01,  1.2110808e+00,  9.9900001e-01,
         -1.4525375e-04,  0.0000000e+00],
        [ 1.1768428e+02,  1.6921492e+01,  1.2060925e+00,  9.9776000e-01,
         -3.2511118e-04,  0.0000000e+00],
        ...,
        [ 1.3663853e+02,  2.9403954e+00,  1.2249781e+00,  1.7549320e+01,
          3.0887530e+00,  0.0000000e+00],
        [ 1.3663853e+02,  2.9403954e+00,  1.2249781e+00,  1.1416820e+01,
          1.4055120e+01,  0.0000000e+00],
        [ 1.3663853e+02,  2.9403954e+00,  1.2249781e+00,  1.7193159e+01,
          7.9130592e+00,  0.0000000e+00]], shape=(5233, 6), dtype=float32),
 'edges': array([[-0.00078404, -0.00098731],
        [ 0.58022213, -0.05370498],
        [-0.03317043,  0.00348765],
        ...,
        [ 0.01863009, -0.02238384],
        [-0.00892198,  0.00269331],
        [-0.01995015, -0.01152593]], shape=(30898, 2

In [11]:
grid[0, 0, 0, :]

array([ 9.9975002e-01, -3.6328966e-05], dtype=float32)

In [12]:
cells[0, 0, 0, :]

array([417,  69, 311], dtype=int32)

In [13]:
data[0, 0, 0, :]

array([1.0473521e+02, 1.3460818e+01, 1.2269930e+00, 1.0143364e+05],
      dtype=float32)

In [14]:
data[0,:10,0,:]

array([[1.04735207e+02, 1.34608183e+01, 1.22699296e+00, 1.01433641e+05],
       [1.12918121e+02, 1.60616550e+01, 1.21108079e+00, 9.98158906e+04],
       [1.17684280e+02, 1.69214916e+01, 1.20609248e+00, 9.92033047e+04],
       [1.20435341e+02, 1.73350391e+01, 1.20139790e+00, 9.86745078e+04],
       [1.22483025e+02, 1.76116867e+01, 1.19768226e+00, 9.82480938e+04],
       [1.24093437e+02, 1.77929859e+01, 1.19450951e+00, 9.78822422e+04],
       [1.25366135e+02, 1.79036484e+01, 1.19175029e+00, 9.75660078e+04],
       [1.26403740e+02, 1.79640598e+01, 1.18937886e+00, 9.72948359e+04],
       [1.27253746e+02, 1.79832287e+01, 1.18738794e+00, 9.70675703e+04],
       [1.27952179e+02, 1.79685001e+01, 1.18575001e+00, 9.68810469e+04]],
      dtype=float32)

In [22]:
from torch_geometric.loader import DataLoader
from utils.graph_dataset import GraphDataset

train_dataset = GraphDataset("/Users/karan94/Documents/MeshGraphNets_Airfoil/data/graphsAll_train_airfoil")
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

for batch in train_loader:
    print(batch)
    break

DataBatch(x=[41864, 7], edge_index=[2, 247184], edge_attr=[247184, 2], y=[41864, 2], batch=[41864], ptr=[9])


In [27]:
import torch
from utils.graph_dataset import GraphDataset
from torch_geometric.loader import DataLoader
from models.meshgraphnet import MeshGraphNet

dataset = GraphDataset("/Users/karan94/Documents/MeshGraphNets_Airfoil/data/graphsAll_train_airfoil")
loader = DataLoader(dataset, batch_size=2, shuffle=False)
batch = next(iter(loader))

print(batch.x.shape[1],
batch.edge_attr.shape[1],
batch.y.shape[1])

7 2 2


In [29]:
# Init model
model = MeshGraphNet(
    node_input_dim=batch.x.shape[1],
    edge_input_dim=batch.edge_attr.shape[1],
    output_dim=batch.y.shape[1],
    latent_dim=128,
    processor_steps=5
)

# Forward pass
out = model(batch)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (61796x384 and 258x128)